In [ ]:
from main.parser.filing_nlp import SpacyFilingTextSearch
from main.parser.filing_nlp_SECU import QuantityRelation, SourceContext, SourceQuantityRelation
from main.parser.parsers import HTMFilingParser
import logging
import pandas as pd
import logging
from spacy.tokens import Token, Span, Doc

In [ ]:
def get_SECU_objects_from_text(text: str):
        search = SpacyFilingTextSearch()
        doc = search.nlp(text)
        secu_objects = search.get_SECU_objects(doc)
        return secu_objects
        
def sample_for_SECU_objects(paths):
        parser = HTMFilingParser()
        search = SpacyFilingTextSearch()
        # create logger file, log file name and log sSECU to info
        exercise_date = []
        exercise_price = []
        expiry = []
        secu_keys = []
        quants = []
        secus = []
        for path in paths:
            text = parser.clean_text_only_filing(parser.get_doc(path))
            doc = search.nlp(text)
            secu_objects = doc._.secu_objects
            secus.append(secu_objects)
            for secu_key, values in secu_objects.items():
                secu_keys.append(secu_key)
                has_exercise_price, has_expiry, has_exercise_date, has_quant = 0, 0, 0, 0
                for secu in values:
                    if secu.exercise_price:
                        has_exercise_price += 1
                    if secu.expiry_date:
                        has_expiry += 1
                    if secu.exercise_date:
                        has_exercise_date += 1
                    if secu.quantity_relations != []:
                        has_quant += len(secu.quantity_relations)
                exercise_date.append(has_exercise_date)
                exercise_price.append(has_exercise_price)
                expiry.append(has_expiry)
                quants.append(has_quant)
        df = pd.DataFrame(data={"name": secu_keys, "quantity_relations": quants ,"exercise_date": exercise_date, "exercise_price": exercise_price, "expiry": expiry})
        return secus, df

In [ ]:
logging.disable(logging.INFO)
filing_paths = [
    r"C:/Users/Olivi/Testing/sec_scraping/tests/test_resources/filings/0000831547/S-3/000083154720000018/cleans-3.htm",
    # r"C:/Users/Olivi/Testing/sec_scraping/tests/test_resources/filings/0001325879/S-3/000119312518218817/d439397ds3.htm",
    r"C:/Users/Olivi/Testing/sec_scraping/tests/test_resources/filings/0001453593/S-3/000149315221008120/forms-3.htm"
]
secus = []
parser = HTMFilingParser()
securities, df = sample_for_SECU_objects(filing_paths)   

In [ ]:
print(df)

In [ ]:
from main.parser.filing_nlp_SECU import SECU
def get_outstanding_relations(self: SECU):
    if self.amods:
        str_secu_amods = [i.lower_ for i in self.amods]
    else:
        str_secu_amods = []
    for qr in self.quantity_relations:
        quant = qr.quantity
        if quant.datetime_relation:
            if quant.original._.negated is False:
                if quant.amods is not None:
                    amods = [i.lower_ for i in quant.amods] + str_secu_amods
                else:
                    amods = str_secu_amods

                if ("outstanding" in amods) or ("outstanding" in str_secu_amods):
                    certainty = 1.0
                    if quant.parent_verb:
                        if quant.parent_verb._.certainty_info:
                            certainty = quant.parent_verb._.certainty_info.determine_level()
                        if certainty > 0.85:
                            if quant.amount.amount is not None:
                                print(self.secu_key)
                                print(quant.original.sent)
                                print(amods, " ", quant.amount, f" [{'negative' if quant.parent_verb._.negated is True else 'positive'}]{quant.parent_verb}@{certainty}")
                            else:
                                print("GOT NONE AMOUNT, DEBUG:")
                                print(quant.original, quant.original.sent)
                                for token in quant.original.sent:
                                    print(token, token.dep_, token.ent_type_)
                                print(amods, " ", quant.amount, f" [{'negative' if quant.parent_verb._.negated is True else 'positive'}]{quant.parent_verb}@{certainty}")

In [1]:
from main.parser.filing_nlp import token_adj_getter
no_amods_count = 0
for d in securities:
    for secu_key, secus in d.items():
        for secu in secus:
            # figure out why it misses some outstanding
            outstanding = secu.get_outstanding_quantity_relations()
            if outstanding:
                for qr in outstanding:
                    print(qr)
            # if secu.quantity_relations:
            #     for qr in secu.quantity_relations:
            #         quant = qr.quantity
            #         certainty = 1.0
            #         if quant.parent_verb:
            #             if quant.parent_verb._.certainty_info:
            #                 certainty = quant.parent_verb._.certainty_info.determine_level()
            #         if quant.amods is not None:
            #             pass
            #             # print(quant.amods, " ", quant.amount, f"[{'negative' if quant.parent_verb._.negated is True else 'positive'}]{quant.parent_verb}@{certainty}")
            #         else:
            #             no_amods_count += 1
            #             # print(quant.amods, secu.amods, " ", quant.amount, f"[{'negative' if quant.parent_verb._.negated is True else 'positive'}]{quant.parent_verb}@{certainty}")
            #             # print(quant.original.sent)



NameError: name 'securities' is not defined

In [3]:
from main.parser.filing_nlp import SpacyFilingTextSearch
text = "On March 31, 2021, we had 10000 shares of common stock outstanding."
# text = "As of March 31, 2021, we had outstanding warrants to purchase 7,532,390 shares of our common stock, and stock options to purchase 2,176,272 shares of our common stock."
# text = "As of April 20, 2022, we had warrants outstanding to issue 20000 shares of common stock at 3.0 $ per share."
# text = "On April 20, 2022, we sold 10000 shares of our common stock at a purchase price of 2 $ per share. On April 20, 2022, we sold 10000 shares of our common stock at a purchase price of 2$ per share."
search = SpacyFilingTextSearch()
doc = search.nlp(text)
for secu_key, secus in doc._.secu_objects.items():
    for secu in secus:
        print(secu.secu_key, "_______")
        qr = secu.quantity_relations[0]
        print(qr.quantity.parent_verb, qr.quantity.amods)
        outstanding = secu.get_outstanding_quantity_relations()
        if len(secu.source_quantity_relations) > 0:
            print(secu.source_quantity_relations[0])    
            print(secu.source_quantity_relations[0].target_secu)
        if outstanding:
            for qr in outstanding:
                print(qr)
        print(secu.attr_matcher._get_at_context(secu.parent_verb))



common stock _______
had None
QuantityRelation(                SECUQuantity(10000.0 COUNT -                	 None -                 	 March 31, 2021 - None)                	 common stock)
[]


In [3]:
from main.parser.filing_nlp_dependency_matcher import SecurityDependencyAttributeMatcher
attr_matcher = SecurityDependencyAttributeMatcher()
secus = doc._.secu_objects
for secu_key, vals in secus.items():
    for secu in vals:
        print(secu.secu_key, secu.original.i)
        print(secu.source_quantity_relations)
        print(secu.quantity_relations)
        # print("source: ", attr_matcher.get_possible_source_quantities(secu.original))
        # print("real: ", attr_matcher.get_quantities(secu.original))
        print("--------------------------")

common stock 12
[]
[QuantityRelation(                SECUQuantity(10000.0 COUNT -                	 None -                 	 April 20, 2022 - None)                	 common stock)]
--------------------------
common stock 35
[]
[QuantityRelation(                SECUQuantity(10000.0 COUNT -                	 None -                 	 April 20, 2022 - None)                	 common stock)]
--------------------------


In [ ]:
'''
Rework QuantityRelation to be created only for directly related quantity and security
add a SECU relation that works like the SourceQuantityRelation but includes both SECUs (source and target)
for example in a phrase as:
    'As of March 31, 2021, we had outstanding warrants to purchase 7,532,390 shares of our common stock and stock options to purchase 2,176,272 shares of our common stock.'
    We should have a SECU which references 'warrants' and one which references 'common stock'
    'warrants' SECU should have:
        no quantity associcated with it
        a datetime relation of March 31, 2021
        amods of outstanding
        a SECURelation to SECU 'common stock'
    first 'common stock' SECU should have:
        a QuantityRelation of 7532390 COUNT with a parent_verb 'purchase'
        no datetime relation
        no amods
    'stock options' SECU should have:
        a datetime relation of Match 31, 2021
        a SECURelation to SECU 'common stock'
    second 'common stock' SECU should have:
        no datetime relation
        a QuantityRelation to 2176272 COUNT with parent verb purchase
    
    
    what we currently have:
    a 'warrants' SECU:
        QuantityRelations referencing both SECUQUANTITY's belonging to the 'common stock' SECUs
        correct datetime_relation
    first 'common stock' SECU:
        with the datetime relation of March 31, 2021
        no QuantityRelation
    second 'common stock' SECU:
        with the datetimerelation of March 31, 2021
        correct QuantityRelation
    a 'stock options' SECU:
        with the correct datetime_relation
        a QuantityRelation belonging to a 'common stock' SECU
        wrong amods of ["common stock"]
    

    HOW TO FIX THIS:
        1) revisite how SECU objects are created
            1.1) create more specific cases for datetime relations
            1.2) create more restrictive rules for quantities associated with a SECU
        1.5) create a map for QuantityRelations built. I can create the source relation in a second pass
            A source relation needs to match on a secuquantity already associated with a none source security through a QuantityRelation

        2) create a SECURelation as a bridge between SECU's. get rid of the SourceQuantityRelation
        Source -> quantity -> SECU, always? always
        OR
        the i always need to check the quantity relations source, and just compare the main and source secu to see if they match?
    
    I should store a relation between two secus based on the parent verb of the QuantityRelation
    to get the correct secuquantities from a source secu i need to go up the the dependency tree and take the first none aux verb?
    can i start pattern matching from above verb?

    verbs to look at:
        purchase
    
    Current todo:
        work on source_secu_secuqunatity patterns [x]
        remove old source qunatity relation setting [x]
        add new source quantity relation and context getting in SECUObjectMapper [x]

        reimplement the extractors according to the changes which 
            come with adding the SECU class
            + add securities_accn_occurence table into equation so we can keep track were we found security 
            -> adjust handlers, commands and add model object [x]
            1) test current state of model and databse [x]
            2) see if Security._security_attributes_object can be accessed after creation of Security and that it isnt present when Security is queried from database
            -> workaround used, converting json to dict when needed [x]
            3) implement Company.get_security_by_attributes [x]
            4) implement handlers, command for SecurityAccnOccurence [x]
            5) write tests for 4) [x]
            
        rework tests for changes above  [x]
        implement outstanding securities into pipeline:
        1) filing_nlp (get outstanding from SECU) [x]
        2) command for security fact outstanding [x]
        3) handler for add_outstanding_securities [x]
        4) BaseHTMExtractor function to form correct model objects to pass to handler [x]
        issues:
            * how can i fetch the correct model.Security from a SECU object
            * is the best way to also cram this onto the doc -> no because we add too much coupling between filing_nlp and model
            --> went for wrapper class with a map of model.Security->SECU
        5) remove old BaseHTMExtractor [x]
        6) run extractor tests and rewrite as needed [x]
        7) fix DependencyAttributeMatcher tests, some issue with hashability and DatetimeRelation
        run all tests 

        8) add tests for SecurityDependencyAttributeMatcher
        9) Think about how to Scope context from a given entity
            coreferee?
            eg: 
        9) add context for prep contexts to verbs (at/through)

        Backburner:
        1) rename .doc attribute on Filing classes so it doesnt create confusion with spacy Doc
    


        


    



'''